In [1]:
import numpy as np
from sympleq.core.paulis import PauliSum, PauliString
from sympleq.core.circuits.target import find_map_to_target_pauli_sum
from sympleq.core.circuits import Gate, Circuit, SWAP
from sympleq.utils import get_linear_dependencies
from sympleq.graph_utils import find_one_permutation, permutation_to_swaps, mapping_key, brute_force_all_permutations, find_swapped_dependent_elements
from sympleq.models import ToricCode, Hadamard_Symmetric_PauliSum, SWAP_symmetric_PauliSum
from scripts.experiments.symmetries.src.pauli import symplectic_pauli_reduction
from scripts.experiments.symmetries.src.permutations import find_automorphisms

In [88]:


toric = False
hadamard_symmetry = True
specific = False
swap_symmetry = False

if toric:
    d = 2

    Nx = 3
    Ny = 3
    periodic = False
    c_x = 1.
    c_z = 1.
    c_g = 1.

    TC = ToricCode(Nx, Ny, c_x, c_z, c_g, periodic)
    H_tc = TC.hamiltonian()
    print(H_tc)
    C_reduce = symplectic_pauli_reduction(H_tc)
    H_red = C_reduce.act(H_tc)
    print(H_red)
    H = H_red[:, 0:-8]  # as here there are 3 Pauli symmetries, should automate this bit
    H.remove_trivial_paulis()
    # print(H)
    H.combine_equivalent_paulis()
elif hadamard_symmetry:
    seed = None
    
    d = 2
    n_qubits = 10
    n_sym_q = 2
    n_paulis = 22
    H, C = Hadamard_Symmetric_PauliSum(n_paulis, n_qubits, n_sym_q, seed=seed)
    H.combine_equivalent_paulis()
elif swap_symmetry:
    d = 2
    n_qubits = 5
    n_paulis = 12
    H = SWAP_symmetric_PauliSum(n_paulis, n_qubits)
    # C_reduce = symplectic_pauli_reduction(H)
    # H = C_reduce.act(H)
    H.combine_equivalent_paulis()
    n_paulis = H.n_paulis()
    H.weights = np.ones(n_paulis, dtype=int)  # set all weights to 1

elif specific:
    d = 2 
    dimensions=[d, d, d, d]
    # p_string = ['x1z0 x1z0 x0z1', 'x0z1 x0z1 x1z1', 'x1z1 x1z1 x1z1', 'x1z1 x1z1 x1z0']
    # p_string = ['x1z0 x0z0', 'x0z0 x1z0', 'x0z1 x0z0', 'x0z0 x0z1', 'x0z1 x0z1'] 
    p_string = ['x0z1 x0z0 x0z1 x0z0',
                'x0z0 x0z1 x0z0 x0z1',
                'x0z1 x0z0 x0z0 x0z0', 
                'x0z0 x0z1 x0z0 x0z0',
                'x0z0 x0z0 x0z1 x0z0', 
                'x0z0 x0z0 x0z0 x0z1',
                'x1z0 x0z0 x0z0 x0z0',
                'x0z0 x1z0 x0z0 x0z0', 
                'x0z0 x0z0 x1z0 x0z0', 
                'x0z0 x0z0 x0z0 x1z0',
                'x0z1 x0z1 x0z0 x0z0',
                'x0z0 x0z0 x0z1 x0z1'] 
    H = PauliSum(p_string, dimensions=dimensions, weights=[1 for _ in range(len(p_string))], standardise=False)
    C = Circuit.from_random(len(dimensions), 100, dimensions=dimensions)
    H = C.act(H)
    # C_reduce = symplectic_pauli_reduction(H)
    # H = C_reduce.act(H) 
else:
    # parameters
    n_paulis = 20
    n_qudits = 8
    d = 2
    n_weights = 1 # number of different weights

    dimensions = [d] * n_qudits 
    # make a random pauli sum
    H = PauliSum.from_random(n_paulis, n_qudits, dimensions, rand_weights=False)

    # this bit makes sections of H have different weights
    weights = np.empty(n_paulis, dtype=int)
    section_size = n_paulis // n_weights
    for i in range(n_weights):
        start = i * section_size
        end = (i + 1) * section_size if i < n_weights - 1 else n_paulis
        weights[start:end] = i + 1
    H.combine_equivalent_paulis()
    H.weights = weights
    # print(H.symplectic())
    


In [89]:
spm = H.symplectic_product_matrix()
# print(H)
print(spm)

spm_automorphisms = find_automorphisms(spm)
print(spm_automorphisms)

[[0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1]
 [0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1]
 [0 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 0 1 0 0 1]
 [1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 1]
 [1 0 1 1 0 0 0 1 1 0 1 0 1 0 1 0 0 1 1 1 0 1]
 [1 0 0 1 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 0 1 0]
 [1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1]
 [1 1 1 1 1 1 1 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
 [1 0 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1]
 [1 1 1 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0]
 [0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 1 1 1]
 [1 0 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1]
 [1 0 1 0 0 1 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0]
 [0 1 0 1 1 0 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0]
 [0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1]
 [0 1 1 0 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1]
 [1 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0]
 [1 1 1 1 1 0

In [90]:
independent_paulis, dependencies = get_linear_dependencies(H.tableau(), d)
print(independent_paulis)
print(dependencies)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 19, 20]
{9: [(0, 1), (1, 1), (7, 1)], 16: [(1, 1), (2, 1), (4, 1), (6, 1), (12, 1), (13, 1)], 18: [(0, 1), (2, 1), (7, 1)], 21: [(0, 1), (1, 1), (6, 1), (10, 1), (11, 1), (12, 1), (14, 1), (15, 1), (17, 1)]}


In [91]:
n_symmetries_found = 0
symmetries = []
for i in range(len(spm_automorphisms)): 
    H_target = H[spm_automorphisms[i].list()]

    H_indep = H[independent_paulis]
    H_t_indep = H_target[independent_paulis]

    F, h, _, _ = find_map_to_target_pauli_sum(H_indep, H_t_indep)
    G = Gate('Symmetry', [i for i in range(H.n_qudits())], F.T, 2, h)
    # print(F)
    # check F
    # H_target.swap_paulis(3, 4)
    # H_target.swap_paulis(7, 9)

    # print(H.tableau())
    # print((H_target.tableau() @ F) % d)
    # print(H.symplectic() @ F % d)
    H_out = G.act(H)

    H_s = H.standard_form()
    H_out_s = H_out.standard_form()
    # print(np.array_equal(H_target.tableau(), H.tableau() @ F % d))
    if np.array_equal(H_s.tableau(), H_out_s.tableau()): 
        n_symmetries_found += 1
        symmetries.append(G)
    # print(H_out)
print('Found', n_symmetries_found, 'symmetries')

Found 4 symmetries


In [92]:
for G in symmetries:
    # print(G.symplectic)
    print(np.array_equal(G.symplectic, np.eye(2 * n_qubits)))

True
False
False
False


In [93]:
print(H_s.tableau()-H_out_s.tableau())
print(H_out_s.tableau())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[1 0 1 1 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 0]
 [1 0 1 0 